# @ 주제 : 한국인 메이저리거의 현지 평가 분석

# 1. 가설
* 트위터 글에 나타난 한국인 메이저리거의 현지 평가가 긍정적인지 부정적인지 판단할 수 있다.

# 2. 연구방법
## 2.1 데이터 수집
* 트위터에서 영문으로 작성된 메이저리거 관련 글을 수집함
* 하지만, 한국인 메이저리거 관련 트윗의 수가 적고 그 중 정서가 드러난 글은 더욱 부족함
* 한국인 메이저리거 뿐만 아닌 메이저리거 전체 선수 관련 트윗을 검색하여 정서가 드러난다고 판단되는 300 document를 추출함
* 해당 document가 긍정적인 정서인 경우 1, 부정적인 정서인 경우 0으로 class를 분류함
* (+추가) 후반부에 SVM 분류기 사용을 위해 class를 긍정적인 정서를 1, 부정적인 정서를 -1로 수정하게 됨

> Code - twitter API를 이용한 트위터 해쉬태그 검색

## 2.2 데이터 처리
* 수집한 document들을 단어 벡터로 변환하는 작업이 필요함
* 단어 벡터 중에서 관사나 이름 명사 등 정서와 관련없는 단어(stop word)를 필터링함
* 이 때, 해쉬태그를 포함한 단어도 정서를 나타내는 경우가 존재하여 해쉬태그 문자가 제거된 형태로 해당 word를 사용


> text parsing(영문자 대소문자로만 이루어진 word 중 길이가 1 이상인 word만 남김)

In [59]:
import numpy as np
import re

docArr = []
classArr = []
regEx = re.compile('[^@a-zA-Z]*') #complie: 정규식을 두 번 이상 사용할 경우 효율을 위함. 영문자 대/소문자를 제외한 문자열을 저장

file = open('data/trainingDoc.txt')
for line in file.readlines():
    urlsRemoved = re.sub('(http[s]?:[/][/]|www.)([a-z]|[A-Z]|[0-9]|[/\.]|[~])*', '', line)
    listOfTokens = regEx.split(urlsRemoved) #영문 대/소문자 이외의 문자를 기준으로 word 분리
    temp = []
    for tok in listOfTokens:
        if len(tok) > 1:
            if tok[0] != '@':            
                temp.append(tok.lower())
    docArr.append(temp)
    #docArr.append([tok.lower() for tok in listOfTokens if tok[0] != '@' and len(tok) > 1]) #길이가 1이상인 word를 소문자 형태로 list에 저장

file = open('data/trainingLabel.txt')
for line in file.readlines():
    curLine = line.strip()
    classArr.append(curLine)

print docArr
print classArr

[['hey', 'you', 'aren', 'the', 'quot', 'satan', 'chicken', 'quot', 'are', 'you'], ['mhmm', 'it', 'is', 'percent', 'true', 'lol', 'and', 'yeah', 'macs', 'are', 'the', 'best', 'for', 'sure'], ['good', 'old', 'classic'], ['through', 'hands', 'the', 'air', 'if', 'true', 'play', 'love', 'it', 'when', 'they', 'call', 'me', 'big', 'poppa'], ['good', 'but', 'tired', 'good', 'night'], ['good', 'morning', 'traci', 'just', 'finished', 'my', 'coffee', 'and', 'devotion', 'and', 'ready', 'for', 'sonday'], ['ve', 'been', 'blocking', 'lot', 'more', 'tweeps', 'lately', 'no', 'image', 'or', 'but', 'just', 'links', 'may', 'be', 'blond', 'but', 'not', 'stupid'], ['just', 'feel', 'like', 'came', 'here', 'at', 'bad', 'time', 'everyone', 'is', 'busy', 'wanna', 'go', 'home'], ['weber', 'state', 'dancers', 'were', 'always', 'stuck', 'on', 'east', 'coast', 'we', 'loved', 'them', 'anyways'], ['hope', 'little', 'oven', 'is', 'clear', 'still', 'see', 'cones', 'out'], ['just', 'fried', 'it', 'up', 'regular', 'and',

> Code - stop word 생성 및 필터링

In [60]:
stopWord = ['manny', 'machado', 'the', 'orioles', 'mlb', 'park', 'maeda', 'kenta', 'dodgers', 'tylercollins'\
            'ladodgers', 'com', 'seunghwanoh', 'seung', 'hwan', 'cardinals', 'stlouis', 'twins', 'debut',\
            'hyunsookim', 'kim', 'hyun', 'soo', 'baltimore', 'korean', 'tyler', 'colins', 'tigers',\
            'keuchel', 'houstonastros', 'giants', 'jepsen', 'braves', 'cubs', 'redSox', 'greinke', 'puig',\
            'molina', 'detroitTigers', 'a', 'an', 'for', 'by', 'and', 'of', 'or', 'to', 'is', 'are', 'has',\
            'have', 'about', 'hyunjinryu', 'hyunjin', 'ryu', 'la', 'byeonghopark', 'byeongho',\
            'park', 'twins', 'minnesota', 'minnesotatwins', 'daeholee', 'daeho', 'lee', 'mariners', 'seattle',\
            'seattlemariners','junghokang', 'jungho', 'kang', 'pirates', 'pittsburgh', 'pittsburghpirtes',\
            'sinsoochoo', 'sinsoo', 'choo', 'rangers', 'texas', 'texasrangers', 'darvish', 'osaka', 'yuDarvish',\
            'rangers', ' ', 'at', 'in', 'with', 'indians', 'royals','seunghwanoh', 'cardinals',\
            'i', 'my', 'me','mine','you','your','you','he','his','him','she','her','her','we','our','us',\
            'they','their','them','it','that','myself','yourself','himself','herself','themselves','itselves']

#입력 받은 document에서 stop word를 제거하는 함수
def filterDoc(docArr):
    print len(docArr)
    wordArr = []
    for i in range(len(docArr)):
        temp = []
        for word in docArr[i]:
            if word not in stopWord: #stop word가 아닌경우만 저장
                temp.append(word)

        wordArr.append(temp)
    
    return wordArr

wordArr = filterDoc(docArr)
print wordArr

10000
[['hey', 'aren', 'quot', 'satan', 'chicken', 'quot'], ['mhmm', 'percent', 'true', 'lol', 'yeah', 'macs', 'best', 'sure'], ['good', 'old', 'classic'], ['through', 'hands', 'air', 'if', 'true', 'play', 'love', 'when', 'call', 'big', 'poppa'], ['good', 'but', 'tired', 'good', 'night'], ['good', 'morning', 'traci', 'just', 'finished', 'coffee', 'devotion', 'ready', 'sonday'], ['ve', 'been', 'blocking', 'lot', 'more', 'tweeps', 'lately', 'no', 'image', 'but', 'just', 'links', 'may', 'be', 'blond', 'but', 'not', 'stupid'], ['just', 'feel', 'like', 'came', 'here', 'bad', 'time', 'everyone', 'busy', 'wanna', 'go', 'home'], ['weber', 'state', 'dancers', 'were', 'always', 'stuck', 'on', 'east', 'coast', 'loved', 'anyways'], ['hope', 'little', 'oven', 'clear', 'still', 'see', 'cones', 'out'], ['just', 'fried', 'up', 'regular', 'fed', 'fam', 'bams', 'eggs', 'rice'], ['re', 'welcone', 'll', 'meet', 'probably', 'tweetup', 'great', 'weekend'], ['hahaha', 'okay', 'let', 'do', 'anyway', 'better',

## 2.3 분류기 생성
* Naive Bayes 분류기와 SVM 분류기의 성능을 각각 비교하여 더 좋은 성능의 분류기를 선택함
* (+ 추가) 본 실험에서 사용된 SVM 분류기는 'Machine Learning in Action' 책의 smo 코드를 기반으로 작성됨
* training 및 test data는 사전확률을 맞추기 위해 300개의 document 중 232개의 document만을 사용함
* 232개의 document 중 210(90%)개를 traing set으로, 나머지 22(10%)개의 한국인 메이저리거 관련 document를 test set으로 사용
* training set은 긍정 105개, 부정 105개. test set은 긍정 15개, 부정 7개 사용

> Naive Bayes 분류기 training

In [61]:
import bayes
reload(bayes)

#training document에 존재하는 유일한 단어들의 list 및 그 갯수
myVocabList = bayes.createVocabList(wordArr)
print len(myVocabList)

trainArr = []
NBclassArr = []

#문서 내에 어휘목록의 단어가 존재하는지 여부를 숫자 벡터로 변환(myVocabList내에 존재하면 1, 존재하지 않으면 0)
for postinDoc in wordArr:
    trainArr.append(bayes.setOfWords2Vec(myVocabList, postinDoc))

#문자열(string) 형태의 class 벡터를 정수(int) 형태로 변환함
for strVec in classArr:
    intVec = int(strVec)
    NBclassArr.append(intVec)

p0V,p1V,pAb=bayes.trainNB0(trainArr,NBclassArr) #Naive Bayes 분류기 training

#training 오류 계산
errorCnt = 0
i = 0
for temp in trainArr:
    predict = bayes.classifyNB(temp,p0V,p1V,pAb) #Naive bayes 분류
    if np.sign(predict) != np.sign(int(NBclassArr[i])):
        errorCnt += 1
    #print 'class is ', NBclassArr[i], 'classified as ', np.sign(predict), '\n'
    i += 1

print 'Total error rate : ', float(errorCnt)/ np.shape(trainArr)[0]

12444
Total error rate :  0.1289


> SVM 분류기 training

In [63]:
import svmMLiA
reload(svmMLiA)

SVMclassArr = []

#SVM 적용을 위해 class를 1 or 0 형태가 아닌 1 or -1 형태로 변환함
for strVec in classArr:
    intVec = int(strVec)
    if intVec == 1:
        SVMclassArr.append(int(1))
    else:
        SVMclassArr.append(int(-1))
        
#b, alphas = svmMLiA.smoP(trainArr, SVMclassArr, 200, 0.001, 1000, ('rbf', 200)) #kernel version
b, alphas = svmMLiA.smoP(trainArr, SVMclassArr, 200, 0.001, 1000) #linear version
print b, alphas

L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H


KeyboardInterrupt: 

In [ ]:
#training 결과를 통해 분류기 d(x) = wx + b 추출
ws = svmMLiA.calcWs(alphas, trainArr, SVMclassArr) #분류기의 w항 계산
#print ws
#print trainMat
trainMat = np.mat(trainArr)
SVMclassMat = np.mat(SVMclassArr).transpose()

svInd = np.nonzero(alphas.A > 0)[0] #서포트 벡터의 index를 리스트로 반환
sVs = trainMat[svInd]
labelSV = SVMclassMat[svInd]

print "there are %d Supprot Vectors" % np.shape(sVs)[0]

In [25]:
#training 오류 계산
errorCnt = 0
m = np.shape(trainMat)[0]
for i in range(m):
    #kernelEval = svmMLiA.kernelTrans(sVs, trainMat[i,:], ('rbf', 200))
    #predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b #kernel version
    predict = trainMat[i] * np.mat(ws) + b #linear version
    if np.sign(predict) != np.sign(int(SVMclassArr[i])):
        errorCnt += 1
    print 'class is ', SVMclassArr[i], 'classified as ', np.sign(predict), '\n'

print 'Total error rate : ', float(errorCnt)/ m

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

c

## 2.4 각 분류기 테스트

> 1) NavieBayes 분류기로 테스트

In [55]:
#test를 위한 입력데이터 전처리
NBtestArr = []
testClassArr = []
file =open('data/testDoc.txt')

for line in file.readlines():
    urlsRemoved = re.sub('(http[s]?:[/][/]|www.)([a-z]|[A-Z]|[0-9]|[/\.]|[~])*', '', line)
    listOfTokens = regEx.split(urlsRemoved) #영문 대/소문자 이외의 문자를 기준으로 word 분리
    NBtestArr.append([tok.lower() for tok in listOfTokens if len(tok) > 1]) #길이가 1이상인 word를 소문자 형태로 list에 저장

    filteredArr = filterDoc(NBtestArr)

file = open('data/testLabel.txt')
for line in file.readlines():
    curLine = line.strip()
    testClassArr.append(curLine)



#test 오류 계산
errorCnt = 0
i = 0
for doc in filteredArr:
    temp = np.array(bayes.setOfWords2Vec(myVocabList, doc))
    classified = bayes.classifyNB(temp,p0V,p1V,pAb) #Naive bayes 분류기 수행
    
    #에러율 계산
    if classified != int(testClassArr[i]):
        errorCnt += 1
    print 'class is ', testClassArr[i], 'classified as ', classified, '\n'
    i += 1
    
print 'Total error rate : ', float(errorCnt)/len(filteredArr)    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


> 2) SVM 분류기로 테스트

In [30]:
#test를 위한 입력데이터 전처리
SVMtestArr = []
for doc in filteredArr:
    temp = np.array(bayes.setOfWords2Vec(myVocabList, doc))
    SVMtestArr.append(temp)    
testMat = np.mat(SVMtestArr)

for i in range(len(testClassArr)):
    if testClassArr[i] == '0' :
        testClassArr[i] = '-1'


#test 오류 계산
errorCnt = 0
m = np.shape(testMat)[0]
for i in range(m):
    #kernelEval = svmMLiA.kernelTrans(sVs, testMat[i,:], ('rbf', 200))
    #predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b #kernel version
    predict = trainMat[i] * np.mat(ws) + b #linear version
    if np.sign(predict) != np.sign(int(testClassArr[i])):
        errorCnt += 1
    print 'class is ', testClassArr[i], 'classified as ', np.sign(predict), '\n'

print 'Total error rate : ', float(errorCnt)/ m

the word: power is not in my Vocabulary!
the word: pitching is not in my Vocabulary!
the word: mph is not in my Vocabulary!
the word: strike is not in my Vocabulary!
the word: reliever is not in my Vocabulary!
the word: koreatimesus is not in my Vocabulary!
the word: relief is not in my Vocabulary!
the word: nickname is not in my Vocabulary!
the word: anything is not in my Vocabulary!
the word: stunning is not in my Vocabulary!
the word: galleryelenash is not in my Vocabulary!
the word: londonartfair is not in my Vocabulary!
the word: impermanence is not in my Vocabulary!
the word: series is not in my Vocabulary!
the word: allmatterwilldecay is not in my Vocabulary!
the word: day is not in my Vocabulary!
the word: day is not in my Vocabulary!
the word: byung is not in my Vocabulary!
the word: entire is not in my Vocabulary!
the word: successful is not in my Vocabulary!
the word: roster is not in my Vocabulary!
the word: byung is not in my Vocabulary!
the word: moon is not in my Vocabul

- <b>linear일 때, 훈련 오류율 0%, 검사오류율 23%, 지지 벡터의 개수 161개</b>
- RBF, 0.1일 때, 훈련 오류율 0%, 검사오류율 77%, 지지 벡터의 개수 209개
- RBF, 1일 때, 훈련 오류율 0%, 검사오류율 27%, 지지 벡터의 개수 209개
- RBF, 5일 때, 훈련 오류율 1%, 검사오류율 36%, 지지 벡터의 개수 177개
- RBF, 10일 때, 훈련 오류율 15%, 검사오류율 63%, 지지 벡터의 개수 166개
- RBF, 50일 때, 훈련 오류율 37%, 검사오류율 77%, 지지 벡터의 개수 121개
- RBF, 100일 때, 훈련 오류율 17%, 검사오류율 68%, 지지 벡터의 개수 116개
- RBF, 200일 때, 훈련 오류율 33%, 검사오류율 77%, 지지 벡터의 개수 110개

### > SVM 분류기 중, 선형 SVM 분류기를 사용했을 때 가장 좋은 분류 결과를 보임

# 3. 결론
#### 22개의 test set을 이용한 분류 결과
- Naive Bayes 분류기: 훈련 오류율 4%, 검사오류율 55%
- 선형 SVM 분류기: 훈련 오류율 0%, 검사오류율 23%

#### 따라서, 가장 좋은 성능을 보이는 선형 SVM 분류기를 본 실험의 분류기로 사용

#### > 선형 SVM 분류기를 사용하여 트위터 상의 메이저리거 관련 글의 정서를 77% 확률로 판단할 수 있다

#### 보완할 점
- 복수/단수 처리나 be동사 처리 등의 Stemming 작업이 필요함
- 더 많은 수의 트위터 글을 사용하여 분류 결과를 비교해 볼 필요가 있음

